COCONUT AGE CLASSFICATION

## IMPORTS

In [1]:
import sys

!{sys.executable} -m pip install opencv-python
!"{sys.executable}" -m pip install opencv-python
%pip install tensorflow
%pip install pandas
%pip install scikoit-learn
!curl https://rclone.org/install.sh | sudo bash
!rclone config
!rclone ls gdrive:



'c:\Users\Mary' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



^C
Note: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scikoit-learn (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for scikoit-learn
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  4734    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
curl: (23) Failure writing output to destination, passed 4734 returned 0
'rclone' is not recognized as an internal or external command,
operable program or batch file.
'rclone' is not re

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
from sklearn.linear_model import LinearRegression



## PREPROCESSING

In [ ]:
# folder path inside Drive
dataset_dir = r"G:\My Drive\THESIS JOURNEY\Coconut Dataset"



# ----------------------------------
# IMAGE PREPROCESSING
#-----------------------------------

img_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),   # Flipping
    layers.RandomRotation(0.2),                     # Rotation
    layers.RandomZoom(0.2),                         # Zooming
    layers.RandomContrast(0.2),                     # Contrast
    layers.RandomTranslation(0.1, 0.1),             # Shifting
    layers.Resizing(224, 224),                      # Resize to MobileNet input size
    layers.Rescaling(1./255, offset=-1),            # Normalization to [-1,1]
    layers.CenterCrop(224, 224)
])

max_count = max(class_counts.values())
print(f"Max sample among classes: {max_count}")  

# Create a balanced dataset by duplicating images in underrepresented classes
for class_name, count in class_count.item():
    class_path = os.path.join(dataset_dir, class_name)
    balanced_class_path = os.path.join(dataset_dir, f"balanced_{class_name}")
    os.makedirs(balanced_class_path, exist_ok=True)       

# List all images in the class directory
pic = [
    os.path.join(class_path, img) for img in os.listdir(class_path)
    if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))
]

# Copy all the original images first
for img in pic:
    shutil.copy(img, balanced_class_path)

# If the class has fewer images than the max count, duplicate images
# Augment it until it will become balanced
while len(os.listdir(balanced_class_path)) < max_count:
    img_to_aug = random.choice(pic)
    # You can add augmentation code here if needed
    shutil.copy(img_to_aug, balanced_class_path)
        
    # Load image
    pil_img = Image.open(img_to_aug).convert("RGB").resize((224, 224))
    arr = np.array(pil_img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    
    # Applying Augmentation
    aug_img = augment(arr, training=True).nmpy()[0]
    aug_img = (aug_img * 255).astype(np.uint8)
    
    # Save augmented image
    aug_img_name = f"aug_{(os.listdir(balanced_class_path))}_{os.path.basename(img_to_aug)}"
    Image.fromarray(aug_img).save(os.path.join(balanced_class_path, aug_img_name))
    
# Combine all balanced class directories into a single balanced dataset directory
balanced_dataset_dir = os.path.join(os.path.dirname(dataset_dir), "balanced_dataset")
os.makedirs(balanced_dataset_dir, exist_ok=True)
for class_name in class_count.keys():
    src = os.path.join(dataset_dir, f"balanced_{class_name}")
    dst = os.path.join(balanced_dataset_dir, class_name)
    shutil.move(src, dst)

print("Balanced dataset created at:", balanced_dataset_dir)
        


Error: The folder 'G:\My Drive\THESIS JOURNEY\Coconut Dataset' does not exist. Please check the path.


In [ ]:


# Load the numerical dataset
df = pd.read_csv('Numerical_Datasets.csv', header=1) 


# ----------------------------------
# NUMERICAL PREPROCESSING
#-----------------------------------

# Step 1: Ensure numeric data
for c in ["HeightTrunk", "11leafscars", "Year"]:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Step 2: Applying the PCA Formula: Age = TrunkHeight / L11 + 5
df["age_formula"] = df["HeightTrunk"] / df["11leafscars"] + 5

# Step 3: Min-Max Scaling
if "Year" in df.columns and df["Year"].notna().any():
    df["actual_age"] = 2025 - df["Year"]

    # Step 4: Calibrate the formula using linear regression
    mask = df["actual_age"].notna() & np.isfinite(df["actual_age"])
    x = df.loc[mask, ["age_formula"]]
    y = df.loc[mask, ["actual_age"]]

    # Step 5: Fit linear regression model
    reg = LinearRegression().fit(x, y)
    a = float(reg.coef_[0])
    b = float(reg.intercept_)
else:
    # Fallback if no year column
    a, b = 1.0, 0.0    

# Step 6: Apply calibrated formula
df["final_age"] = a * df["age_formula"] + b

# Steo 7: Clip negative aged or unrealistic ages
df["final_age"] = df["final_age"].clip(lower=0)




# Step 5: Show the results
#print("Calibrated a, b:", a, b)
print(df[["age_formula","actual_age","final_age"]].head(200))
#print(df[['HeightTrunk','11leafscars']].head(10))
#print(df[['HeightTrunk','11leafscars']].describe())
#print(df[['age_formula']].head(10))
#print(df[['final_age']].head(10))


     age_formula  actual_age  final_age
0      20.233333          18  44.217924
1      20.233333          18  44.217924
2      20.233333          18  44.217924
3      19.061538          18  41.045622
4      19.061538          18  41.045622
..           ...         ...        ...
195    25.594595          49  58.732018
196    25.594595          49  58.732018
197    22.720930          49  50.952387
198    22.720930          49  50.952387
199    22.720930          49  50.952387

[200 rows x 3 columns]


C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_13884\88821929.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(reg.coef_[0])
C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_13884\88821929.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b = float(reg.intercept_)


## TRAINING THE MODEL


### CNN-MOBILE NET


### SVM CLASSIFICATION 
